In [9]:
import tensorflow as tf
import torch
import torch.nn as nn
from model_unet import Unet
from torch import onnx

In [17]:
th_model_p = Unet(4,10)
th_model_p = nn.DataParallel(th_model_p)
th_model_p.load_state_dict(torch.load('./models/Unet/best-cosloss.pth'))
th_model_p.eval()
try:
    state_dict = th_model_p.module.state_dict()
except AttributeError:
    state_dict = th_model_p.state_dict()

th_model = Unet(4,10)
th_model.load_state_dict(state_dict)
th_model.eval()
th_model.cuda()

Unet(
  (encoder): ModuleList(
    (0): Conv1d(1, 10, kernel_size=(15,), stride=(1,), padding=(7,))
    (1): Conv1d(10, 20, kernel_size=(15,), stride=(1,), padding=(7,))
    (2): Conv1d(20, 30, kernel_size=(15,), stride=(1,), padding=(7,))
    (3): Conv1d(30, 40, kernel_size=(15,), stride=(1,), padding=(7,))
  )
  (decoder): ModuleList(
    (0): Conv1d(80, 40, kernel_size=(5,), stride=(1,), padding=(2,))
    (1): Conv1d(70, 30, kernel_size=(5,), stride=(1,), padding=(2,))
    (2): Conv1d(50, 20, kernel_size=(5,), stride=(1,), padding=(2,))
    (3): Conv1d(30, 10, kernel_size=(5,), stride=(1,), padding=(2,))
  )
  (ebatch): ModuleList(
    (0): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): BatchNorm1d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): BatchNorm1d(30, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): BatchNorm1d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
 

In [19]:
dummy_input = torch.randn(1, 192, 1, device='cuda')

input_names = [ "actual_input_1" ]
output_names = [ "output1" ]

onnx._export(th_model, dummy_input, "./Unet.onnx", verbose=True, input_names=input_names, output_names=output_names)

graph(%actual_input_1 : Float(1, 192, 1)
      %1 : Float(10, 1, 15)
      %2 : Float(10)
      %3 : Float(20, 10, 15)
      %4 : Float(20)
      %5 : Float(30, 20, 15)
      %6 : Float(30)
      %7 : Float(40, 30, 15)
      %8 : Float(40)
      %9 : Float(40, 80, 5)
      %10 : Float(40)
      %11 : Float(30, 70, 5)
      %12 : Float(30)
      %13 : Float(20, 50, 5)
      %14 : Float(20)
      %15 : Float(10, 30, 5)
      %16 : Float(10)
      %17 : Float(10)
      %18 : Float(10)
      %19 : Float(10)
      %20 : Float(10)
      %21 : Long()
      %22 : Float(20)
      %23 : Float(20)
      %24 : Float(20)
      %25 : Float(20)
      %26 : Long()
      %27 : Float(30)
      %28 : Float(30)
      %29 : Float(30)
      %30 : Float(30)
      %31 : Long()
      %32 : Float(40)
      %33 : Float(40)
      %34 : Float(40)
      %35 : Float(40)
      %36 : Long()
      %37 : Float(40)
      %38 : Float(40)
      %39 : Float(40)
      %40 : Float(40)
      %41 : Long()
      %42 : Float(30)


RuntimeError: ONNX export failed: Couldn't export operator aten::upsample_linear1d

Defined at:
/opt/conda/lib/python3.7/site-packages/torch/nn/functional.py(2439): interpolate
/opt/conda/lib/python3.7/site-packages/torch/nn/functional.py(2352): upsample
/workspace/SpeechToEGG/model_unet.py(168): forward
/opt/conda/lib/python3.7/site-packages/torch/nn/modules/module.py(477): _slow_forward
/opt/conda/lib/python3.7/site-packages/torch/nn/modules/module.py(487): __call__
/opt/conda/lib/python3.7/site-packages/torch/jit/__init__.py(251): forward
/opt/conda/lib/python3.7/site-packages/torch/nn/modules/module.py(489): __call__
/opt/conda/lib/python3.7/site-packages/torch/jit/__init__.py(196): get_trace_graph
/opt/conda/lib/python3.7/site-packages/torch/onnx/utils.py(192): _trace_and_get_graph_from_model
/opt/conda/lib/python3.7/site-packages/torch/onnx/utils.py(224): _model_to_graph
/opt/conda/lib/python3.7/site-packages/torch/onnx/utils.py(281): _export
/opt/conda/lib/python3.7/site-package